In [ ]:
# | default_exp layers/embeddings

# Imports

In [ ]:
# | export

from typing import Union

import numpy as np
import torch
from einops import rearrange, repeat
from torch import nn

from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field, model_validator
from vision_architectures.utils.normalizations import get_norm_layer

# Config

In [ ]:
# | export


class RelativePositionEmbeddings3DConfig(CustomBaseModel):
    num_heads: int = Field(..., description="Number of query attention heads")
    grid_size: tuple[int, int, int]

    @property
    def num_patches(self) -> int:
        return np.prod(self.grid_size)

    @model_validator(mode="before")
    @classmethod
    def validate_before(cls, data):
        grid_size = data.get("grid_size")
        if isinstance(data["grid_size"], int):
            data["grid_size"] = (grid_size, grid_size, grid_size)
        return data

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        if isinstance(self.grid_size, int):
            self.grid_size = (self.grid_size, self.grid_size, self.grid_size)
        return self


class AbsolutePositionEmbeddings3DConfig(CustomBaseModel):
    dim: int
    grid_size: tuple[int, int, int] | None = None
    learnable: bool = False

    @property
    def num_patches(self) -> int:
        return np.prod(self.grid_size)

    @model_validator(mode="before")
    @classmethod
    def validate_before(cls, data):
        if isinstance(data.get("grid_size"), int):
            data["grid_size"] = (
                data["grid_size"],
                data["grid_size"],
                data["grid_size"],
            )
        return data

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        if isinstance(self.grid_size, int):
            self.grid_size = (self.grid_size, self.grid_size, self.grid_size)

        if self.learnable and self.grid_size is None:
            raise ValueError("grid_size must be provided if learnable is True")
        return self


class PatchEmbeddings3DConfig(CustomBaseModel):
    patch_size: tuple[int, int, int]
    in_channels: int
    dim: int
    norm_layer: str = "layernorm"

# Architecture

### Position Embeddings

In [ ]:
# | export


def get_coords_grid(grid_size: tuple[int, int, int]) -> torch.Tensor:
    d, h, w = grid_size

    grid_d = torch.arange(d, dtype=torch.int32)
    grid_h = torch.arange(h, dtype=torch.int32)
    grid_w = torch.arange(w, dtype=torch.int32)

    grid = torch.meshgrid(grid_d, grid_h, grid_w, indexing="ij")
    grid = torch.stack(grid, axis=0)
    # (3, d, h, w)

    return grid

In [ ]:
# | export


class RelativePositionEmbeddings3D(nn.Module):
    def __init__(self, config: RelativePositionEmbeddings3DConfig = {}, **kwargs):
        super().__init__()

        self.config = RelativePositionEmbeddings3DConfig.model_validate(config | kwargs)

        num_heads = self.config.num_heads
        grid_size = self.config.grid_size

        # TODO: Add embed_spacing_info functionality

        relative_limits = (
            2 * grid_size[0] - 1,
            2 * grid_size[1] - 1,
            2 * grid_size[2] - 1,
        )

        self.relative_position_bias_table = nn.Parameter(torch.randn(num_heads, np.prod(relative_limits)))
        # (num_heads, num_patches_z * num_patches_y * num_patches_x)

        # Pair-wise relative position index for each token inside the window
        coords = get_coords_grid(grid_size)
        coords_flatten = rearrange(coords, "three d h w -> three (d h w)", three=3)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()
        relative_coords[:, :, 0] += grid_size[0] - 1
        relative_coords[:, :, 1] += grid_size[1] - 1
        relative_coords[:, :, 2] += grid_size[2] - 1
        relative_position_index: torch.Tensor = (
            relative_coords[:, :, 0] * relative_limits[1] * relative_limits[2]
            + relative_coords[:, :, 1] * relative_limits[2]
            + relative_coords[:, :, 2]
        )
        self.relative_position_index = relative_position_index.flatten()
        # (num_patches, num_patches)

    def forward(self):
        relative_position_embeddings = self.relative_position_bias_table[:, self.relative_position_index]
        # (num_heads, num_patches, num_patches)
        relative_position_embeddings = relative_position_embeddings.reshape(
            1, self.config.num_patches, self.config.num_patches, -1
        )
        # (1, num_patches, num_patches, num_heads)
        relative_position_embeddings = relative_position_embeddings.permute(0, 3, 1, 2).contiguous()
        # (1, num_heads, num_patches, num_patches)
        return relative_position_embeddings

In [ ]:
test = RelativePositionEmbeddings3D(num_heads=6, grid_size=4)
display(test)
display(test().shape)

RelativePositionEmbeddings3D()

torch.Size([1, 6, 64, 64])

In [ ]:
# | export


class RelativePositionEmbeddings3DMetaNetwork(nn.Module):
    def __init__(self, config: RelativePositionEmbeddings3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = RelativePositionEmbeddings3DConfig.model_validate(config | kwargs)

        num_heads = self.config.num_heads
        grid_size = self.config.grid_size

        # TODO: Add embed_spacing_info functionality
        self.cpb_mlp = nn.Sequential(
            nn.Linear(3, 512, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_heads, bias=False),
        )

        relative_limits = (
            2 * grid_size[0] - 1,
            2 * grid_size[1] - 1,
            2 * grid_size[2] - 1,
        )

        # Relative coordinates table
        relative_coords_table = get_coords_grid(relative_limits).float()
        for i in range(3):
            relative_coords_table[i] = (relative_coords_table[i] - (grid_size[0] - 1)) / (
                grid_size[0] - 1 + 1e-8  # small value added to ensure there is no NaN when window size is 1
            )
        relative_coords_table = rearrange(
            relative_coords_table,
            "three num_patches_z num_patches_y num_patches_x -> num_patches_z num_patches_y num_patches_x three",
        ).contiguous()
        relative_coords_table *= 8  # Normalize to -8, 8
        relative_coords_table = (
            torch.sign(relative_coords_table) * torch.log2(torch.abs(relative_coords_table) + 1.0) / np.log2(8)
        )
        # (num_patches_z, num_patches_y, num_patches_x, 3)
        # Allow moving this to and from cuda whenever required but don't save to state_dict
        self.register_buffer("relative_coords_table", relative_coords_table, persistent=False)

        # Pair-wise relative position index for each token inside the window
        coords = get_coords_grid(grid_size)
        coords_flatten = rearrange(coords, "three d h w -> three (d h w)", three=3)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()
        relative_coords[:, :, 0] += grid_size[0] - 1
        relative_coords[:, :, 1] += grid_size[1] - 1
        relative_coords[:, :, 2] += grid_size[2] - 1
        relative_position_index: torch.Tensor = (
            relative_coords[:, :, 0] * relative_limits[1] * relative_limits[2]
            + relative_coords[:, :, 1] * relative_limits[2]
            + relative_coords[:, :, 2]
        )
        self.relative_position_index = relative_position_index.flatten()
        # (num_patches, num_patches)

        self.checkpointing_level1 = ActivationCheckpointing(1, checkpointing_level)

    def get_relative_position_embeddings_table(self):
        # (num_patches_z, num_patches_y, num_patches_x, 3)
        relative_position_embeddings_table: torch.Tensor = self.cpb_mlp(self.relative_coords_table)
        # (num_patches_z, num_patches_y, num_patches_x, num_heads)
        relative_position_embeddings_table = relative_position_embeddings_table.reshape(-1, self.config.num_heads)
        # (num_patches, num_heads)
        return relative_position_embeddings_table

    def forward(self):
        relative_position_embeddings_table = self.checkpointing_level1(self.get_relative_position_embeddings_table)
        # (num_patches, num_heads)
        relative_position_embeddings = relative_position_embeddings_table[self.relative_position_index]
        # (num_patches * num_patches, num_heads)
        relative_position_embeddings = rearrange(
            relative_position_embeddings,
            "(num_patches1 num_patches2) num_heads -> num_heads num_patches1 num_patches2",
            num_patches1=self.config.num_patches,
            num_patches2=self.config.num_patches,
            num_heads=self.config.num_heads,
        ).contiguous()
        # (num_heads, num_patches, num_patches)
        relative_position_embeddings = 16 * torch.sigmoid(relative_position_embeddings)
        # (num_heads, num_patches, num_patches)
        return relative_position_embeddings

In [ ]:
test = RelativePositionEmbeddings3DMetaNetwork(num_heads=6, grid_size=(4, 4, 4))
display(test)
display(test().shape)


RelativePositionEmbeddings3DMetaNetwork(
  (cpb_mlp): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=6, bias=False)
  )
)

torch.Size([6, 64, 64])

In [ ]:
# | export

RelativePositionEmbeddings = Union[RelativePositionEmbeddings3D, RelativePositionEmbeddings3DMetaNetwork]

In [ ]:
# | export


def get_absolute_position_embeddings_3d(
    dim: int,
    grid_size: tuple[int, int, int],
    spacing: tuple[float, float, float] = (1.0, 1.0, 1.0),
    # Used if the embeddings required are of a crop of a larger image
    full_image_size: tuple[int, int, int] = None,
    crop_center: tuple[int, int, int] = None,
) -> torch.Tensor:
    if dim % 6 != 0:
        raise ValueError("embed_dim must be divisible by 6")

    if full_image_size is None:
        full_image_size = grid_size

    grid = get_coords_grid(grid_size)
    # (3, d, h, w)

    # Apply offset if crop parameters are provided
    if crop_center is not None:
        # Calculate crop boundaries
        crop_start = tuple(max(0, center - size // 2) for center, size in zip(crop_center, grid_size))

        # Offset the grid coordinates to represent their position in the full volume
        for i in range(3):
            grid[i] = grid[i] + crop_start[i]

    grid = rearrange(grid, "x d h w -> x 1 d h w")
    # (3, 1, d, h, w)

    omega = torch.arange(dim // 6, dtype=torch.float32)
    omega /= dim / 6.0
    omega = 1.0 / 10000**omega
    # (dim // 6)

    patch_multiplier = torch.Tensor(spacing) / min(spacing)

    position_embeddings = []
    for i, grid_subset in enumerate(grid):
        grid_subset = grid_subset.reshape(-1)

        out = torch.einsum("m,d->md", grid_subset, omega)

        emb_sin = torch.sin(out)
        emb_cos = torch.cos(out)

        emb = torch.cat([emb_sin, emb_cos], axis=1) * patch_multiplier[i]
        position_embeddings.append(emb)

    position_embeddings = torch.cat(position_embeddings, axis=1)
    # (dim, d * h * w)
    position_embeddings = rearrange(
        position_embeddings,
        "(d h w) e -> 1 e d h w",
        d=grid_size[0],
        h=grid_size[1],
        w=grid_size[2],
    )
    # (1, dim, d, h, w)

    return position_embeddings

In [ ]:
# | export


class AbsolutePositionEmbeddings3D(nn.Module):
    def __init__(self, config: AbsolutePositionEmbeddings3DConfig = {}, **kwargs):
        super().__init__()

        self.config = AbsolutePositionEmbeddings3DConfig.model_validate(config | kwargs)

        dim = self.config.dim
        grid_size = self.config.grid_size
        learnable = self.config.learnable

        # TODO: Add getting absolute position embeddings of subsection of the grid

        self.position_embeddings_cache = {}
        self.position_embeddings = None
        if grid_size is not None:
            self.position_embeddings_cache[grid_size] = get_absolute_position_embeddings_3d(dim, grid_size)
            self.position_embeddings = nn.Parameter(self.position_embeddings_cache[grid_size], requires_grad=learnable)

    def forward(
        self,
        batch_size=None,
        grid_size=None,
        spacings: torch.Tensor = None,
        device=torch.device("cpu"),
        # Used if the embeddings required are of a crop of a larger image
        full_image_size: tuple[int, int, int] = None,
        crop_center: tuple[int, int, int] = None,
    ):
        assert self.position_embeddings is not None or grid_size is not None, "grid_size must be provided"
        assert batch_size is not None or spacings is not None, "Either batch_size or spacings must be provided"

        if isinstance(grid_size, int):
            grid_size = (grid_size, grid_size, grid_size)

        if self.position_embeddings is not None:
            position_embeddings = self.position_embeddings
        else:
            cache_key = (grid_size, full_image_size, crop_center)
            if cache_key not in self.position_embeddings_cache:
                self.position_embeddings_cache[cache_key] = get_absolute_position_embeddings_3d(
                    self.config.dim, grid_size, full_image_size=full_image_size, crop_center=crop_center
                )
            position_embeddings = self.position_embeddings_cache[cache_key].to(device)
        # (1, dim, d, h, w)

        if batch_size is not None:
            b = batch_size
        else:
            assert spacings.ndim == 2 and spacings.shape[1] == 3, "spacings must be of shape (batch_size, 3)"
            assert self.config.dim % 3 == 0, "embed_dim must be divisible by 3"

            b = spacings.shape[0]

        position_embeddings = repeat(position_embeddings, "1 e d h w -> b e d h w", b=b)

        if spacings is not None:
            # (b, 3)
            spacings = repeat(
                spacings,
                "b three -> b (three dim_by_three) 1 1 1",
                three=3,
                dim_by_three=self.config.dim // 3,
            )
            # (b, dim, 1, 1, 1)

            position_embeddings = position_embeddings * spacings.to(position_embeddings.device)
            # (b, dim, d, h, w)

        return position_embeddings

In [ ]:
test = AbsolutePositionEmbeddings3D(dim=6, grid_size=4, learnable=True)
display(test)
display(test(batch_size=2).shape)
display(test(spacings=torch.randn(4, 3)).shape)

test = AbsolutePositionEmbeddings3D(dim=6)
display(test(grid_size=(3, 3, 3), batch_size=2).shape)

AbsolutePositionEmbeddings3D()

torch.Size([2, 6, 4, 4, 4])

torch.Size([4, 6, 4, 4, 4])

torch.Size([2, 6, 3, 3, 3])

### Patch embeddings

In [ ]:
# | export


class PatchEmbeddings3D(nn.Module):
    def __init__(self, config: PatchEmbeddings3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = PatchEmbeddings3DConfig.model_validate(config | kwargs)

        patch_size = self.config.patch_size
        in_channels = self.config.in_channels
        dim = self.config.dim
        norm_layer = self.config.norm_layer

        self.patch_embeddings = nn.Conv3d(
            in_channels=in_channels,
            out_channels=dim,
            kernel_size=patch_size,
            stride=patch_size,
        )

        if norm_layer is None:
            self.normalization = nn.Identity()
        elif isinstance(norm_layer, nn.Module):
            self.normalization = norm_layer(dim)
        else:
            self.normalization = get_norm_layer(norm_layer, dim)

        self.checkpointing_level1 = ActivationCheckpointing(1, checkpointing_level)

    def _forward(self, pixel_values: torch.Tensor):
        # pixel_values: (b, c, z, y, x)

        embeddings = self.patch_embeddings(pixel_values)
        # (b, dim, num_patches_z, num_patches_y, num_patches_x)
        embeddings = rearrange(embeddings, "b d z y x -> b z y x d")
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)
        embeddings = self.normalization(embeddings)
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)
        embeddings = rearrange(embeddings, "b z y x d -> b d z y x")
        # (b, dim, num_patches_z, num_patches_y, num_patches_x)

        return embeddings

    def forward(self, pixel_values: torch.Tensor):
        return self.checkpointing_level1(self._forward, pixel_values)

In [ ]:
test = PatchEmbeddings3D(patch_size=(1, 8, 8), in_channels=1, dim=12)
display(test)
o = test(torch.randn(2, 1, 32, 512, 512))
display(o.shape)


PatchEmbeddings3D(
  (patch_embeddings): Conv3d(1, 12, kernel_size=(1, 8, 8), stride=(1, 8, 8))
  (normalization): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
)

torch.Size([2, 12, 32, 64, 64])

# nbdev

In [ ]:
!nbdev_export

# Rough work